In [ ]:
#pip install pymssql

In [1]:
import pymssql
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from pymssql import Connection
import pandas as pd


In [2]:
# sql server engine func
def get_engine(username:str, password:str, server:str, database:str) -> Engine:
    """ connects to MS SQL Server
    Args:
        username (str)
        password (str)
        server (str)
        database (str)
    Returns:
        Engine
    """
    conn_string = f"mssql+pymssql://{username}:{password}@{server}/{database}"
    engine = create_engine(conn_string)
    return engine

# get connection
def get_cnx(username:str, password:str, server:str, database:str) -> Connection:
    """ enables querying MS SQL Server
    Args:
        username (str): [description]
        password (str): [description]
        server (str): [description]
        database (str): [description]
    Returns:
        pymssql.Connection: [description]
    """
    cnx = pymssql.connect(user=username, password=password,
                          host=server, database=database)
    return cnx

# read table to pandas
def read_sql_table(table_name:str, cnx:pymssql.Connection) -> pd.DataFrame:
    """reads sql table into pandas df
    Args:
        table_name (str): table name in SQL DB
        cnx (pymssql.Connection): connection object used to read query
    Returns:
        pd.DataFrame: [description]
    """
    sql_table = pd.read_sql(sql=f"SELECT * FROM {table_name}",
                            con=cnx)

    return sql_table

# write df tp sql server
def write_df_to_sql(
        expID: str, new_df: pd.DataFrame, table_name: str, cnx: Connection, engine: Engine,
        replace_exist_table=False):
    """[summary]
    #columns_to_consider: List[str]
    Args:
        new_df (pd.DataFrame): name of df to be written to sql db
        table_name (str): name of table to write to 
        cnx (Connection): used to read sql tables
        engine (Engine): used to write to sql db 
        columns_to_consider (List[str]): columns to be examined when checking for duplicates
        init (bool, optional): indicates if tables need 
        to be created for the first time. Defaults to False.
    """
    # add expID
    new_df.insert(0,'ExpID',str(expID))

    # conert all columns to string
    new_df=new_df.applymap(str)

    # convert all columns named with (min) to _min
    new_df.columns=[i.replace('(min)','_min') if '(min)' in i else i for i in new_df.columns]
    
    #write new_df to sql server
    if replace_exist_table:
        if_exists='replace'
    
    else:  
        if_exists='append'

    new_df.to_sql(name=table_name,con=engine, index=False, if_exists=if_exists)
    


In [3]:
engine = get_engine("transam1","Pass@0@0!","usodcvsql0255", "NAO_PMO_Analytics")

In [4]:
connection = get_cnx("transam1","Pass@0@0!","usodcvsql0255", "NAO_PMO_Analytics")

In [5]:
# Read the input data already sitting on SQL 
data = read_sql_table("zzz_Forecast_Input",connection)

In [6]:
# Export this data to CSV to use in R
data.to_csv("sql_input.csv")

In [7]:
# Come back later when we need to export the final results from R
exportsql = pd.read_csv("sql_export.csv")
exportsql.head()

Date                                  Scenario Product Sub.Product  \
0  2019-01-01  Field Ops - Mass Markets - Test 7/7/2021    EAST       FL_TX   
1  2019-02-01  Field Ops - Mass Markets - Test 7/7/2021    EAST       FL_TX   
2  2019-03-01  Field Ops - Mass Markets - Test 7/7/2021    EAST       FL_TX   
3  2019-04-01  Field Ops - Mass Markets - Test 7/7/2021    EAST       FL_TX   
4  2019-05-01  Field Ops - Mass Markets - Test 7/7/2021    EAST       FL_TX   

   Volumes  c1_AAA_ets  c1_AAN_ets  c1_add_holtlineartrend  \
0    52567         0.0         0.0                     0.0   
1    45137         0.0         0.0                     0.0   
2    50092         0.0         0.0                     0.0   
3    51663         0.0         0.0                     0.0   
4    46423         0.0         0.0                     0.0   

   c1_add_holtlineartrenddamp  c1_ANA_ets  ...  \
0                         0.0         0.0  ...   
1                         0.0         0.0  ...   
2                         0.0         0.0  ...   
3                         0.0         0.0  ...   
4                         0.0         0.0  ...   

   c1_triple_holtwinters_multiplicative90  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   c1_triple_holtwinters_multiplicative91  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   c1_triple_holtwinters_multiplicative92  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   c1_triple_holtwinters_multiplicative93  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   c1_triple_holtwinters_multiplicative94  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   c1_triple_holtwinters_multiplicative95  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   c1_triple_holtwinters_multiplicative96  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   c1_triple_holtwinters_multiplicative97  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   c1_triple_holtwinters_multiplicative98  \
0                                     0.0   
1                                     0.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   

   c1_triple_holtwinters_multiplicative99  
0                                     0.0  
1                                     0.0  
2                                     0.0  
3                                     0.0  
4                                     0.0  

[5 rows x 309 columns]

In [8]:
# Writing the final output from R to SQL as a table
write_df_to_sql("t", exportsql, "zzz_chris_final", connection ,  engine,
        replace_exist_table=True)